In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import os
import time

# ======= Step 1: Set up environment =======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EXPORT_DIR = "exported_weights"
os.makedirs(EXPORT_DIR, exist_ok=True)

# ======= Step 2: Define data loaders =======
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# ======= Step 3: Define BNN model =======
class BNN(nn.Module):
    def __init__(self):
        super(BNN, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.sign(self.bn1(self.fc1(x)))
        x = self.fc2(x)
        return x

model = BNN().to(device)

# ======= Step 4: Helper functions =======
def fold_bn_into_fc(fc, bn):
    w = fc.weight.data
    b = fc.bias.data if fc.bias is not None else torch.zeros_like(bn.running_mean)
    gamma = bn.weight.data
    beta = bn.bias.data
    mean = bn.running_mean
    var = bn.running_var
    eps = bn.eps
    std = torch.sqrt(var + eps)
    w_folded = w * (gamma / std).unsqueeze(1)
    b_folded = (b - mean) / std * gamma + beta
    return w_folded, b_folded

def export_ternary_weights(weight_tensor, filename):
    with open(os.path.join(EXPORT_DIR, filename), 'w') as f:
        for row in weight_tensor:
            encoded = []
            for w in row:
                val = w.item()
                if val > 0:
                    encoded.append("1")
                elif val < 0:
                    encoded.append("0")
                else:
                    encoded.append("2")  # pruned
            f.write(''.join(encoded) + '\n')

def measure_inference_time(model, test_loader, label=""):
    model.eval()
    correct = 0
    start_time = time.time()
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            pred = outputs.argmax(dim=1)
            correct += (pred == labels).sum().item()
    end_time = time.time()
    total_time = end_time - start_time
    avg_time = total_time / len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset) * 100

    print(f"\n🕒 {label} Inference Results")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Total Inference Time: {total_time:.4f} seconds")
    print(f"Average Time per Image: {avg_time * 1000:.4f} ms")
    return total_time, avg_time

# ======= Step 5: Train with flipping frequency tracking =======
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
flip_count = torch.zeros_like(model.fc1.weight.data).int().to(device)
prev_sign = torch.sign(model.fc1.weight.data.clone().detach())

for epoch in range(10):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            current_sign = torch.sign(model.fc1.weight.data)
            flip = (current_sign != prev_sign).int()
            flip_count += flip
            prev_sign = current_sign.clone()

    # Evaluate accuracy
    model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            pred = outputs.argmax(dim=1)
            correct += (pred == labels).sum().item()

    accuracy = correct / len(test_dataset)
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {accuracy*100:.2f}%")
    if epoch == 9:
        measure_inference_time(model, test_loader, label="Before Pruning")

# ======= Step 6: Prune based on flipping frequency =======
flattened = flip_count.view(-1).float()
num_to_prune = int(flattened.numel() * 0.2)
_, prune_indices = torch.topk(flattened, num_to_prune, largest=True)
mask = torch.ones_like(flattened)
mask[prune_indices] = 0
mask = mask.view_as(model.fc1.weight.data)

with torch.no_grad():
    model.fc1.weight.data *= mask
print("✂️ Pruned unstable weights based on flipping frequency.")

# ======= Step 7: Fine-tune after pruning =======
print("🔁 Fine-tuning pruned model...")
for epoch in range(3):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Final test
model.eval()
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        pred = outputs.argmax(dim=1)
        correct += (pred == labels).sum().item()

final_acc = correct / len(test_dataset)
print(f"✅ Final Accuracy After Pruning and Fine-Tuning: {final_acc*100:.2f}%")
measure_inference_time(model, test_loader, label="After Pruning")

# ======= Step 8: Export ternary weights =======
fc1_folded_w, _ = fold_bn_into_fc(model.fc1, model.bn1)
export_ternary_weights(fc1_folded_w, "fc1_weights_ternary.mem")
export_ternary_weights(model.fc2.weight.data, "fc2_weights_ternary.mem")
print("📦 Exported ternary .mem weights.")


Epoch 1, Loss: 0.8075, Accuracy: 88.15%
Epoch 2, Loss: 0.2448, Accuracy: 88.65%
Epoch 3, Loss: 0.3648, Accuracy: 88.65%
Epoch 4, Loss: 0.3521, Accuracy: 88.69%
Epoch 5, Loss: 0.5038, Accuracy: 88.80%
Epoch 6, Loss: 0.2842, Accuracy: 88.96%
Epoch 7, Loss: 0.3332, Accuracy: 88.84%
Epoch 8, Loss: 0.3343, Accuracy: 88.87%
Epoch 9, Loss: 0.2547, Accuracy: 88.78%
Epoch 10, Loss: 0.3792, Accuracy: 88.88%

🕒 Before Pruning Inference Results
Accuracy: 88.88%
Total Inference Time: 5.9919 seconds
Average Time per Image: 0.5992 ms
✂️ Pruned unstable weights based on flipping frequency.
🔁 Fine-tuning pruned model...
✅ Final Accuracy After Pruning and Fine-Tuning: 85.09%

🕒 After Pruning Inference Results
Accuracy: 85.09%
Total Inference Time: 6.0150 seconds
Average Time per Image: 0.6015 ms
📦 Exported ternary .mem weights.
